In [3]:
from sklearn import tree
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re


In [4]:
import spacy
from spacy.lang.en import English
import contextualSpellCheck

from sklearn.base import BaseEstimator, TransformerMixin
from string import punctuation
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier

In [5]:
train_data = pd.read_csv('nlp-getting-started/train.csv', index_col=0)
test_data = pd.read_csv('nlp-getting-started/test.csv', index_col=0)

In [6]:
train_data.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
print(train_data.shape, test_data.shape)

(7613, 4) (3263, 3)


In [8]:
train_data.isnull().sum()

keyword       61
location    2533
text           0
target         0
dtype: int64

## Dealing with mislabeling

In [9]:
mislabeled_data = train_data.groupby(['text']).nunique().sort_values(by='target', ascending=False)
mislabeled_data = mislabeled_data[mislabeled_data['target'] > 1]['target']
mislabeled_data.index.tolist()

['like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit',
 'Hellfire! We don\x89Ûªt even want to think about it or mention it so let\x89Ûªs not do anything that leads to it #islam!',
 "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'",
 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!',
 'To fight bioterrorism sir.',
 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE',
 '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption',
 '#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect',
 'He came to a land which was engulfed in tribal war and turned it into a land 

In [10]:
def fix_target(l):
    true_labels = [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
    for i in range(len(l)):
        indexes = train_data.loc[train_data['text'] == l[i]].index
        for index in indexes:
            train_data.loc[index, 'target'] = true_labels[i]           

In [11]:
l = mislabeled_data.index.tolist()
fix_target(l)

## Separate Text

In [29]:
text = pd.concat([train_data.text, test_data.text])

In [30]:
text.head()

id
1    Our Deeds are the Reason of this #earthquake M...
4               Forest fire near La Ronge Sask. Canada
5    All residents asked to 'shelter in place' are ...
6    13,000 people receive #wildfires evacuation or...
7    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [14]:
nlp = English()  # Includes English data
nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")


## Transformers

In [80]:
from sklearn.base import BaseEstimator, TransformerMixin
from string import punctuation

In [92]:
def remove_username(text):
    pattern = '@[/w]*'
    return [word for word in text if not re.match(pattern, word.text)] 

def remove_links(text):
    pattern = "(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)"
    return [word for word in text if not re.match(pattern, word.text)] 

def text_to_token(text):
    return nlp(text)

def remove_digits(text):
    pattern = '^[0-9]*([,.][0-9]*)?$'
    return [word for word in text if not re.match(pattern, word.text)] 

def remove_stopwords(text):
    stopwords = nlp.Defaults.stop_words
    return [word for word in text if word not in stopwords] 

def remove_short_words(text):
    return [word for word in text if len(word) > 3]

def remove_punctuation(text):
    return [word for word in text if word.text not in punctuation and word.text not in ['\n']]

def lemmatize(text):
    return[token.lemma_ for token in text]

def list_to_str(l):
    return ' '.join(l)

In [87]:
class TextTokenizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self ):
        print("Tokenize")

    def fit(self, X):
        return self    
        
    def transform(self, X):
        text = X.apply(text_to_token)
        return text 

In [86]:
class TextCleanerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self ):
        print("Clean text")

    def fit(self, X):
        return self    
        
    def transform(self, X):
        text = X.apply(remove_digits)
        text = text.apply(remove_stopwords)
        text = text.apply(remove_punctuation)
        text = text.apply(remove_username)
        text = text.apply(remove_links)
        text = text.apply(remove_short_words)
        return text 

In [99]:
class TextLemmatizerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self ):
        print("Lemmatize")

    def fit(self, X):
        return self    
        
    def transform(self, X):
        lemmatized_text = X.apply(lemmatize)
        lemmatized_text = lemmatized_text.apply(remove_stopwords)
        text = lemmatized_text.apply(list_to_str)
        return text 

In [100]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('tokenize', TextTokenizerTransformer()),
    ('clean', TextCleanerTransformer()),
    ('lemmatize', TextLemmatizerTransformer())
])

Tokenize
Clean text
Lemmatize


In [101]:
tmp = pipe.fit_transform(text)

In [102]:
tmp[1]

'deed Reason earthquake ALLAH forgive'

### Spelling correction


In [24]:
from autocorrect import Speller

In [25]:
spell = Speller()

In [117]:
# import contextualSpellCheck

In [134]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# contextualSpellCheck.add_to_pipe(nlp)
# doc = nlp('Income was $9.4 milion compared to the prior year of $2.7 milion.')

# print(doc._.performed_spellCheck) #Should be True
# print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.

True
Income was $9.4 million compared to the prior year of $2.7 million.


In [121]:
# nlp.add_pipe('sentencizer')

In [245]:
def correct_spelling(text):
    return [spell(word) for word in text]

In [249]:
text[117:118].apply(correct_spelling)

id
170    [I, seeing, Issues, aftershock]
Name: text, dtype: object

In [251]:
for i in range(len(text)):
#     text[i] = text[i].apply(correct_spelling)
    text[i] = correct_spelling(text[i])
    if i % 10 == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [273]:
text_with_spelling = text.copy()

In [272]:
text.head()

id
1    [Our, Deeds, Reason, earthquake, May, ALL, For...
4        [Forest, fire, near, La, Range, Task, Canada]
5    [All, residents, asked, shelter, place, notifi...
6    [13,000, people, receive, wildfires, evacuatio...
7    [Just, got, sent, photo, Ruby, Alaska, smoke, ...
Name: text, dtype: object

In [255]:
text = text.apply(remove_stopwords)

In [70]:
text.head()

id
1           [deed, Reason, earthquake, ALLAH, forgive]
4                   forest fire near Ronge Sask Canada
5    resident shelter place notify officer evacuati...
6    [13,000, people, receive, wildfire, evacuation...
7    [send, photo, Ruby, Alaska, smoke, wildfires, ...
Name: text, dtype: object

### Lemmatization

In [223]:
l = text[3]
# l = train_data.text
# doc = nlp()# running runner")
print([token.lemma_ for token in l])

['there', 'forest', 'fire', 'spot', 'pond', 'geese', 'flee', 'across', 'street', 'save', 'they']


In [102]:
def lemmatize(text):
    return[token.lemma_ for token in text]

In [224]:
lemmatized_text = text.apply(lemmatize)

## From lemma back to token

In [107]:
text

id
1               [deed, Reason, earthquake, ALLAH, forgive]
4                [forest, fire, near, Ronge, Sask, Canada]
5        [resident, ask, shelter, place, notify, office...
6        [13,000, people, receive, wildfire, evacuation...
7        [send, photo, Ruby, Alaska, smoke, wildfires, ...
                               ...                        
10861    [earthquake, SAFETY, ANGELES, SAFETY, fastener...
10865    [storm, bad, hurricane, city&amp;3other, hard,...
10868                   [Green, Line, derailment, Chicago]
10874                 [issue, Hazardous, Weather, Outlook]
10875    [CityofCalgary, activate, Municipal, Emergency...
Name: text, Length: 10876, dtype: object

In [105]:
text = text.apply(remove_stopwords)

In [106]:
def list_to_tokens(text):
    return [nlp(word) for word in text]
#     token_list = []
# #     text = nlp(text)
#     for token in text:
#         token_list.append(nlp(token)#.text)
#     return token_list  

In [108]:
token_text = text.apply(list_to_tokens)

In [109]:
token_text.head()

id
1    [(deed), (Reason), (earthquake), (ALLAH), (for...
4    [(forest), (fire), (near), (Ronge), (Sask), (C...
5    [(resident), (ask), (shelter), (place), (notif...
6    [(13,000), (people), (receive), (wildfire), (e...
7    [(send), (photo), (Ruby), (Alaska), (smoke), (...
Name: text, dtype: object

## Vectorizing

In [505]:
def vectorize(line):
    return [word.vector for word in line]
    

In [525]:
token_text[0:5].apply(vectorize)

id
1    [[-0.027740717, -0.11559065, 0.14615542, -0.11...
4    [[3.0948722, 0.90669054, 0.21063933, -0.480993...
5    [[-1.2147622, 0.49894887, 0.28072032, -0.30333...
6    [[0.5988686, 2.0787547, 0.22975582, -0.5023366...
7    [[-0.5375247, 1.4861925, -0.7751549, 0.3462769...
Name: text, dtype: object

In [526]:
vec_text = token_text.apply(vectorize)

## Vectorization using TF-IDF

In [243]:
from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()
# tfidf = str_text.apply(tfidf_vectorizer.fit_transform)
# tfidf.shape

In [190]:
def list_to_str(l):
    return ' '.join(l)

In [225]:
str_text = lemmatized_text.apply(list_to_str)

In [226]:
str_text

id
1                deed Reason this earthquake ALLAH forgive
4                       forest fire near Ronge Sask Canada
5        resident ask shelter place be notify officer o...
6        people receive wildfire evacuation order Calif...
7        just send this photo from Ruby Alaska smoke fr...
                               ...                        
10861       earthquake SAFETY ANGELES SAFETY fastener XrWn
10865    storm bad than last hurricane city&amp;3other ...
10868                        Green Line derailment Chicago
10874                      issue Hazardous Weather Outlook
10875    CityofCalgary activate Municipal Emergency Pla...
Name: text, Length: 10876, dtype: object

In [234]:
df = pd.DataFrame(str_text)

In [248]:
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=10000)


In [249]:
tfidf_vect.fit(str_text)
tfidf = tfidf_vect.transform(str_text)

In [250]:
tfidf

<10876x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 87344 stored elements in Compressed Sparse Row format>

In [252]:
print(str_text[0], tfidf[0])

just happen terrible crash   (0, 8246)	0.2133565265618833
  (0, 5819)	0.36498017243543707
  (0, 2882)	0.46371919693647873
  (0, 2432)	0.34375279192218894
  (0, 2107)	0.5419728924830057
  (0, 235)	0.4408314582176946


In [469]:
# train_vec = tfidf[0:len(train_data)]
# test_vec = tfidf[len(train_data):]

## Train-Test

In [253]:
train_vec = tfidf[0:len(train_data)]
test_vec = tfidf[len(train_data):]

In [254]:
y = train_data.target
X = train_vec

In [255]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Train

In [256]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

/Users/yulialysenko/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:50:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [258]:
prediction_xgb = xgb.predict(X_test)

In [259]:
from sklearn.metrics import f1_score
f1_score(y_test,prediction_xgb)

0.6929217668972859

In [2]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    #loss_function='CrossEntropy'
)


# clf.fit(X_train, y_train, 

# )

In [1]:
clf.fit(X_train, y_train)

NameError: name 'clf' is not defined